In [48]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [49]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


In [50]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


In [51]:
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)


In [52]:
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)


In [53]:
train_size = int(0.85 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size


In [54]:
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])


In [55]:
val_dataset.dataset.transform = test_transform


In [56]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)


In [57]:
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)


ANN MODEL

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
def train_model(model, train_loader, val_loader, epochs=10):
    device = torch.device("cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # Validation
        model.eval()
        total_val_loss = 0
        correct, total = 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        train_losses.append(total_train_loss / len(train_loader))
        val_losses.append(total_val_loss / len(val_loader))
        val_accuracies.append(100 * correct / total)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_losses[-1]:.4f}, "
              f"Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.2f}%")

    return train_losses, val_losses, val_accuracies


In [18]:
model = SimpleANN()
train_loss, val_loss, val_acc = train_model(model, train_loader, val_loader, epochs=7)


Epoch [1/7], Train Loss: 1.6568, Val Loss: 1.5397, Val Acc: 46.49%
Epoch [2/7], Train Loss: 1.4505, Val Loss: 1.5011, Val Acc: 47.61%
Epoch [3/7], Train Loss: 1.3321, Val Loss: 1.4802, Val Acc: 48.88%
Epoch [4/7], Train Loss: 1.2413, Val Loss: 1.4025, Val Acc: 51.51%
Epoch [5/7], Train Loss: 1.1549, Val Loss: 1.4122, Val Acc: 52.11%
Epoch [6/7], Train Loss: 1.0768, Val Loss: 1.4492, Val Acc: 50.69%
Epoch [7/7], Train Loss: 0.9983, Val Loss: 1.4274, Val Acc: 52.17%


In [19]:
model = SimpleANN()
train_loss, val_loss, val_acc = train_model(model, train_loader, val_loader, epochs=12)

Epoch [1/12], Train Loss: 1.6554, Val Loss: 1.5298, Val Acc: 45.47%
Epoch [2/12], Train Loss: 1.4494, Val Loss: 1.4522, Val Acc: 49.17%
Epoch [3/12], Train Loss: 1.3389, Val Loss: 1.4104, Val Acc: 50.77%
Epoch [4/12], Train Loss: 1.2348, Val Loss: 1.4123, Val Acc: 51.25%
Epoch [5/12], Train Loss: 1.1572, Val Loss: 1.3897, Val Acc: 52.40%
Epoch [6/12], Train Loss: 1.0693, Val Loss: 1.4267, Val Acc: 52.49%
Epoch [7/12], Train Loss: 0.9976, Val Loss: 1.4318, Val Acc: 52.95%
Epoch [8/12], Train Loss: 0.9247, Val Loss: 1.4778, Val Acc: 53.89%
Epoch [9/12], Train Loss: 0.8357, Val Loss: 1.5453, Val Acc: 52.57%
Epoch [10/12], Train Loss: 0.7690, Val Loss: 1.6058, Val Acc: 52.35%
Epoch [11/12], Train Loss: 0.7052, Val Loss: 1.6799, Val Acc: 52.81%
Epoch [12/12], Train Loss: 0.6415, Val Loss: 1.7731, Val Acc: 52.39%


In [20]:
model = SimpleANN()
train_loss, val_loss, val_acc = train_model(model, train_loader, val_loader, epochs=20)

Epoch [1/20], Train Loss: 1.6539, Val Loss: 1.5103, Val Acc: 46.36%
Epoch [2/20], Train Loss: 1.4450, Val Loss: 1.4340, Val Acc: 49.89%
Epoch [3/20], Train Loss: 1.3363, Val Loss: 1.4175, Val Acc: 50.72%
Epoch [4/20], Train Loss: 1.2420, Val Loss: 1.3897, Val Acc: 52.40%
Epoch [5/20], Train Loss: 1.1571, Val Loss: 1.4199, Val Acc: 51.45%
Epoch [6/20], Train Loss: 1.0721, Val Loss: 1.4363, Val Acc: 52.15%
Epoch [7/20], Train Loss: 0.9959, Val Loss: 1.4110, Val Acc: 53.47%
Epoch [8/20], Train Loss: 0.9170, Val Loss: 1.4616, Val Acc: 53.11%
Epoch [9/20], Train Loss: 0.8472, Val Loss: 1.5235, Val Acc: 52.36%
Epoch [10/20], Train Loss: 0.7744, Val Loss: 1.5987, Val Acc: 53.00%
Epoch [11/20], Train Loss: 0.7020, Val Loss: 1.7154, Val Acc: 53.09%
Epoch [12/20], Train Loss: 0.6431, Val Loss: 1.8126, Val Acc: 52.39%
Epoch [13/20], Train Loss: 0.5972, Val Loss: 1.9043, Val Acc: 52.57%
Epoch [14/20], Train Loss: 0.5443, Val Loss: 1.9878, Val Acc: 51.87%
Epoch [15/20], Train Loss: 0.5020, Val Loss

In [13]:
def eval_u(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for x, y in dataloader:
            outputs = model(x)
            _, preds = torch.max(outputs, 1)
            y_true.extend(y.numpy())
            y_pred.extend(preds.numpy())

    from sklearn.metrics import classification_report
    print("Classification Report:")
    print(classification_report(y_true, y_pred, digits=4))


In [29]:
eval_u(model,test_loader)

Classification Report:
              precision    recall  f1-score   support

           0     0.6092    0.6080    0.6086      1000
           1     0.6983    0.5740    0.6301      1000
           2     0.4773    0.3470    0.4019      1000
           3     0.3674    0.3850    0.3760      1000
           4     0.4534    0.4620    0.4577      1000
           5     0.4102    0.4410    0.4251      1000
           6     0.5627    0.6100    0.5854      1000
           7     0.5412    0.6300    0.5823      1000
           8     0.6809    0.5740    0.6229      1000
           9     0.5270    0.6430    0.5793      1000

    accuracy                         0.5274     10000
   macro avg     0.5328    0.5274    0.5269     10000
weighted avg     0.5328    0.5274    0.5269     10000



VGG-16

In [58]:
from torchvision import models

In [59]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [17]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False  # Freeze convolution layers

vgg16.classifier[6] = nn.Linear(4096, 10)  # Change output layer to 10 classes


c:\Users\Rakshita\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Rakshita\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.classifier.parameters(), lr=0.0005)


In [19]:
train_losses, val_losses, val_accuracies = [], [], []

for epoch in range(5):
    vgg16.train()
    train_loss = 0

    for images, labels in train_loader:
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    vgg16.eval()
    val_loss = 0
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = vgg16(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    train_losses.append(train_loss / len(train_loader))
    val_losses.append(val_loss / len(val_loader))
    val_accuracies.append(100 * correct / total)

    print(f"Epoch {epoch+1}: Train Loss={train_losses[-1]:.4f}, "
          f"Val Loss={val_losses[-1]:.4f}, Val Acc={val_accuracies[-1]:.2f}%")


Epoch 1: Train Loss=1.3422, Val Loss=1.1072, Val Acc=61.31%
Epoch 2: Train Loss=1.2073, Val Loss=1.0330, Val Acc=64.71%
Epoch 3: Train Loss=1.1604, Val Loss=1.0377, Val Acc=65.25%
Epoch 4: Train Loss=1.1157, Val Loss=1.0051, Val Acc=65.45%
Epoch 5: Train Loss=1.0826, Val Loss=1.0091, Val Acc=65.79%


In [20]:
eval_u(vgg16,test_loader)

Classification Report:
              precision    recall  f1-score   support

           0     0.7261    0.6390    0.6798      1000
           1     0.6731    0.8070    0.7340      1000
           2     0.6318    0.5250    0.5735      1000
           3     0.5046    0.6090    0.5519      1000
           4     0.6327    0.5460    0.5862      1000
           5     0.7080    0.5190    0.5990      1000
           6     0.5848    0.8070    0.6782      1000
           7     0.7047    0.6490    0.6757      1000
           8     0.7341    0.7400    0.7371      1000
           9     0.6871    0.6720    0.6795      1000

    accuracy                         0.6513     10000
   macro avg     0.6587    0.6513    0.6495     10000
weighted avg     0.6587    0.6513    0.6495     10000



VGG 19

In [21]:
vgg19 = models.vgg19(pretrained=True)

for param in vgg19.features.parameters():
    param.requires_grad = False


vgg19.classifier[6] = nn.Linear(4096, 10)


c:\Users\Rakshita\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Rakshita\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [41]:
def train(model, train_loader, val_loader, epochs=7):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

    train_losses, val_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for x, y in train_loader:
            outputs = model(x)
            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                outputs = model(x)
                _, preds = torch.max(outputs, 1)
                correct += (preds == y).sum().item()
                total += y.size(0)

        acc = 100 * correct / total
        train_losses.append(total_loss / len(train_loader))
        val_accuracies.append(acc)

        print(f"Epoch {epoch+1}: Train Loss = {train_losses[-1]:.4f}, Val Acc = {acc:.2f}%")

    return train_losses, val_accuracies


In [22]:
train_losses, val_accuracies = train(vgg19, train_loader, val_loader, epochs=7)


Epoch 1: Train Loss = 1.5927, Val Acc = 54.19%
Epoch 2: Train Loss = 1.4915, Val Acc = 57.41%
Epoch 3: Train Loss = 1.4342, Val Acc = 60.87%
Epoch 4: Train Loss = 1.4107, Val Acc = 58.08%
Epoch 5: Train Loss = 1.3842, Val Acc = 61.39%
Epoch 6: Train Loss = 1.3488, Val Acc = 60.77%
Epoch 7: Train Loss = 1.3190, Val Acc = 62.32%
